In [16]:
!pip install isbnlib

Python(95112) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [17]:
import isbnlib
import pandas as pd
import requests

In [18]:
isbn_df = pd.read_csv('final_isbn_data_with_filled_values.csv')

In [19]:
isbn_df.count()

ISBN-10    35741200
ISBN-13    38756968
dtype: int64

In [20]:
# check nan
isbn_df.isnull().sum()

ISBN-10    3135864
ISBN-13     120096
dtype: int64

In [21]:
isbn_df.dropna(inplace=True)

In [22]:
#find nan
isbn_df.isnull().sum()



ISBN-10    0
ISBN-13    0
dtype: int64

Randomly select 100 rows from the dataframe

In [23]:
isbn_df = isbn_df.sample(n=100)
isbn_10 = isbn_df['ISBN-10'].tolist()

In [24]:
# count the number of rows
isbn_df.count()

ISBN-10    100
ISBN-13    100
dtype: int64

## Fetch Metadata Using ISBNLib
Use the `isbnlib.meta()` function to fetch metadata for each ISBN. The function returns a dictionary with metadata information. If the ISBN is not found, the function returns an empty dictionary.

In [25]:
def fetch_metadata(isbn, service="goob"):
    try:
        metadata = isbnlib.meta(isbn, service=service)
        return metadata if metadata else {}
    except Exception as e:
        print(f"Error fetching metadata for ISBN {isbn} using service {service}: {e}")
        return {}

In [26]:
def normalize_metadata(isbn_df, services):
    for service in services:
        isbn_df[f"{service}_metadata"] = isbn_df['ISBN-10'].apply(lambda x: fetch_metadata(x, service=service))
        normalized = pd.json_normalize(isbn_df[f"{service}_metadata"])
        normalized.columns = [f"{service}_{col}" for col in normalized.columns]
        isbn_df = pd.concat([isbn_df, normalized], axis=1)
        isbn_df.drop(columns=[f"{service}_metadata"], inplace=True)

    return isbn_df

In [27]:
isbn_df['ISBN-10']

9575640     1534664726
21158276    0815628129
19786266    1794862188
37931203    8494761102
23836774    2900768632
               ...    
19382413    140242275X
26965237    331948088X
36903485    066424162X
14728440    0597746095
23831256    0070049165
Name: ISBN-10, Length: 100, dtype: object

In [28]:
services = ["goob", "openl","wiki"]
isbn_df = normalize_metadata(isbn_df, services)
isbn_df.head()


Error fetching metadata for ISBN 1398604100 using service goob: isbn request != isbn response (9781398604100 not in [{'type': 'ISBN_13', 'identifier': '9781398604094'}, {'type': 'ISBN_10', 'identifier': '1398604097'}])
Error fetching metadata for ISBN 9040200580 using service goob: isbn request != isbn response (9789040200588 not in [{'type': 'ISBN_13', 'identifier': '9789025970406'}, {'type': 'ISBN_10', 'identifier': '9025970400'}])
Error fetching metadata for ISBN 1453897704 using service goob: an HTTP error has ocurred ((503) Service Unavailable)
Error fetching metadata for ISBN 2080674161 using service goob: an HTTP error has ocurred ((503) Service Unavailable)
Error fetching metadata for ISBN 0822363607 using service goob: isbn request != isbn response (9780822363606 not in [{'type': 'ISBN_13', 'identifier': '9780822373056'}, {'type': 'ISBN_10', 'identifier': '082237305X'}])
Error fetching metadata for ISBN nan using service openl: 'float' object is not iterable
Error fetching met

,ISBN-10,ISBN-13,goob_ISBN-13,goob_Title,goob_Authors,goob_Publisher,goob_Year,goob_Language,openl_ISBN-13,openl_Title,openl_Authors,openl_Publisher,openl_Year,openl_Language,wiki_ISBN-13,wiki_Title,wiki_Authors,wiki_Publisher,wiki_Year,wiki_Language
9575640,1534664726,9.781535e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21158276,0815628129,9.780816e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19786266,1794862188,9.781795e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37931203,8494761102,9.788495e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23836774,2900768632,9.782901e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


it is a dictionary, so we need to convert it to a dataframe

## Google Books Data

In [29]:
def get_book_data(isbn):
    url = f"https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn}"
    response = requests.get(url)
    data = response.json()

    if "items" in data:
        book = data["items"][0]["volumeInfo"]
        return {
            "ISBN": isbn,
            "Title": book.get("title", "N/A"),
            "Authors": ", ".join(book.get("authors", ["Unknown"])),
            "Published Date": book.get("publishedDate", "N/A"),
            "Description": book.get("description", "No description available"),
            "Page Count": book.get("pageCount", "N/A"),
            "Categories": ", ".join(book.get("categories", ["Unknown"])),
            "Thumbnail": book.get("imageLinks", {}).get("thumbnail", "")
        }
    else:
        return {"ISBN": isbn, "Title": "Not Found"}

In [30]:
books_data = [get_book_data(isbn) for isbn in isbn_10]

books_df = pd.DataFrame(books_data)

In [31]:
books_df

,ISBN,Title,Authors,Published Date,Description,Page Count,Categories,Thumbnail
0,1534664726,Not Found,NaN,NaN,NaN,NaN,NaN,NaN
1,0815628129,Making Peace Prevail,Alice Ackermann,2000-01-01,The urgency to tell the story of the former Yu...,238,History,http://books.google.com/books/content?id=oqEkc...
2,1794862188,Modernist Myth,Nanette Norris,2021-11-02,The writers in this study were living at the e...,178,Unknown,http://books.google.com/books/content?id=qpvOz...
3,8494761102,Not Found,NaN,NaN,NaN,NaN,NaN,NaN
4,2900768632,Not Found,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
95,140242275X,Not Found,NaN,NaN,NaN,NaN,NaN,NaN
96,331948088X,Plants and Health,"Elizabeth Anne Olson, John Richard Stepp",2016-12-29,This volume showcases current ethnobiological ...,183,Science,http://books.google.com/books/content?id=i3nQD...
97,066424162X,Not Found,NaN,NaN,NaN,NaN,NaN,NaN
98,0597746095,Not Found,NaN,NaN,NaN,NaN,NaN,NaN


## Goodreads Data


In [66]:
import os
print(os.getcwd())

os.environ['MallocStackLogging'] = '0'


/Users/wuyuetong/Documents/ASCoR Intership/Invisible book


In [67]:
!mkdir classic_book_metadata

zsh(95544) MallocStackLogging: could not tag MSL-related memory as no_footprint, so those pages will be included in process footprint - (null)
mkdir(95544) MallocStackLogging: could not tag MSL-related memory as no_footprint, so those pages will be included in process footprint - (null)
mkdir: classic_book_metadata: File exists


Python(95544) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [84]:
# Select the first 100 rows of the 'ISBN-13' column and convert to a list
isbn_13 = isbn_df['ISBN-13'].head(100).tolist()
print(isbn_13)

['9781534664722.0', '9780815628125.0', '9781794862180.0', '9788494761102.0', '9782900768631.0', '9780590819336.0', '9789864855438.0', '9788416428571.0', '9786587043234.0', '9782711833825.0', '9780191531972.0', '9780963456748.0', '9783552033306.0', '9781546215271.0', '9787301136294.0', '9781317363484.0', '9788416094660.0', '9780500342954.0', '9781644928363.0', '9786053324492.0', '9783788706647.0', '9780618591145.0', '9781139087193.0', '9788981075163.0', '9781410814715.0', '9780444641052.0', '9780072831535.0', '9780415592314.0', '9788172201470.0', '9780312988111.0', '9781015359321.0', '9781401394424.0', '9780070546448.0', '9781013558917.0', '9781305039940.0', '9781461052661.0', '9783930612574.0', '9781081316396.0', '9780342099979.0', '9781581180787.0', '9781511495219.0', '9780850451856.0', '9780711038325.0', '9781475776010.0', '9781398604100.0', '9781451597790.0', '9780692672105.0', '9780618934669.0', '9781448895472.0', '9781322871059.0', '9788495119025.0', '9780739478844.0', '9781304784

In [86]:
# Clean the list by stripping newline characters and removing the trailing .0
cleaned_isbn_list = [isbn.strip().replace('.0', '') for isbn in isbn_13]

# Save the cleaned list to a text file
with open('cleaned_isbn_list.txt', 'w') as f:
    for isbn in cleaned_isbn_list:
        f.write(f"{isbn}\n")

In [96]:
# read the cleaned list
with open('cleaned_isbn_list.txt', 'r') as f:
    isbn_list = f.readlines()
print(isbn_list)
# change to integer
isbn_list = [int(isbn) for isbn in isbn_list]
with open('isbn_list.txt', 'w') as f:
    for isbn in isbn_list:
        f.write(f"{isbn}\n")

['9781534664722\n', '9780815628125\n', '9781794862180\n', '9788494761102\n', '9782900768631\n', '9780590819336\n', '9789864855438\n', '9788416428571\n', '9786587043234\n', '9782711833825\n', '9780191531972\n', '9780963456748\n', '9783552033306\n', '9781546215271\n', '9787301136294\n', '9781317363484\n', '9788416094660\n', '9780500342954\n', '9781644928363\n', '9786053324492\n', '9783788706647\n', '9780618591145\n', '9781139087193\n', '9788981075163\n', '9781410814715\n', '9780444641052\n', '9780072831535\n', '9780415592314\n', '9788172201470\n', '9780312988111\n', '9781015359321\n', '9781401394424\n', '9780070546448\n', '9781013558917\n', '9781305039940\n', '9781461052661\n', '9783930612574\n', '9781081316396\n', '9780342099979\n', '9781581180787\n', '9781511495219\n', '9780850451856\n', '9780711038325\n', '9781475776010\n', '9781398604100\n', '9781451597790\n', '9780692672105\n', '9780618934669\n', '9781448895472\n', '9781322871059\n', '9788495119025\n', '9780739478844\n', '9781304784

In [97]:
!python get_books.py --isbn_list_path isbn_list.txt --output_directory_path classic_book_metadata --format csv

zsh(95714) MallocStackLogging: could not tag MSL-related memory as no_footprint, so those pages will be included in process footprint - (null)
python(95714) MallocStackLogging: could not tag MSL-related memory as no_footprint, so those pages will be included in process footprint - (null)
Python(95714) MallocStackLogging: could not tag MSL-related memory as no_footprint, so those pages will be included in process footprint - (null)


Python(95714) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


ISBNs to scrape: ['9781534664722', '9780815628125', '9781794862180', '9788494761102', '9782900768631', '9780590819336', '9789864855438', '9788416428571', '9786587043234', '9782711833825', '9780191531972', '9780963456748', '9783552033306', '9781546215271', '9787301136294', '9781317363484', '9788416094660', '9780500342954', '9781644928363', '9786053324492', '9783788706647', '9780618591145', '9781139087193', '9788981075163', '9781410814715', '9780444641052', '9780072831535', '9780415592314', '9788172201470', '9780312988111', '9781015359321', '9781401394424', '9780070546448', '9781013558917', '9781305039940', '9781461052661', '9783930612574', '9781081316396', '9780342099979', '9781581180787', '9781511495219', '9780850451856', '9780711038325', '9781475776010', '9781398604100', '9781451597790', '9780692672105', '9780618934669', '9781448895472', '9781322871059', '9788495119025', '9780739478844', '9781304784759', '9781791326609', '9781622296613', '9780821342169', '9780701124328', '978970722357

In [98]:
goodreads_df = pd.read_csv('classic_book_metadata/all_books.csv')
goodreads_df

,isbn,book_url,title,author,authorlink,average_rating,num_pages,genres,publication_info,format,rating_distribution,cover_image_uri,book_details
0,9781534664722,https://www.goodreads.com/search?q=9781534664722,Bead Tapestry Patterns Peyote Hubble Good Eart...,Georgia Grisolia,https://www.goodreads.com/author/show/4377463....,0.00,['152'],[],"['Published June 14, 2016']","['152 pages, Paperback']","{'5': '0', '4': '0', '3': '0', '2': '0', '1': ...",https://images-na.ssl-images-amazon.com/images...,A book of patterns for making Tapestry Peyote ...
1,9780815628125,https://www.goodreads.com/search?q=9780815628125,Making Peace Prevail: Preventing Violent Confl...,Alice Ackermann,https://www.goodreads.com/author/show/536765.A...,0.00,['272'],[],"['First published February 1, 1999']","['272 pages, Hardcover']","{'5': '0', '4': '0', '3': '0', '2': '0', '1': ...",https://images-na.ssl-images-amazon.com/images...,The urgency to tell the story of the former Yu...
2,9781794862180,https://www.goodreads.com/search?q=9781794862180,"Modernist Myth: Studies in H.D., D.H. Lawrence...",Nanette Norris,https://www.goodreads.com/author/show/3406785....,0.00,['177'],[],"['First published September 22, 2008']","['177 pages, Paperback']","{'5': '0', '4': '0', '3': '0', '2': '0', '1': ...",https://images-na.ssl-images-amazon.com/images...,The writers in this study were living at the e...
3,9788494761102,https://www.goodreads.com/search?q=9788494761102,LATIDOS DE VALDEDIOS,cscj Madre Olga María del Redentor,https://www.goodreads.com/author/show/21152169...,0.00,['275'],[],"['Published October 1, 2017']","['275 pages, Paperback']","{'5': '0', '4': '0', '3': '0', '2': '0', '1': ...",https://images-na.ssl-images-amazon.com/images...,NaN
4,9782900768631,https://www.goodreads.com/search?q=9782900768631,Dizains disette,Jean-Pierre Georges,https://www.goodreads.com/author/show/25907876...,0.00,['125'],[],"['Published January 1, 1987']","['125 pages, Paperback']","{'5': '0', '4': '0', '3': '0', '2': '0', '1': ...",https://images-na.ssl-images-amazon.com/images...,French
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,9780140245219,https://www.goodreads.com/search?q=9780140245219,I Remember,Joe Brainard,https://www.goodreads.com/author/show/19989.Jo...,4.19,['146'],[],"['First published January 1, 1970']","['146 pages, Paperback']","{'5': '1,198', '4': '903', '3': '437', '2': '1...",https://images-na.ssl-images-amazon.com/images...,In a book which uniquely captures 1950's Ameri...
68,9782406106197,https://www.goodreads.com/search?q=9782406106197,Les Corps de Marivaux (L'Europe Des Lumieres),Clemence Aznavour,https://www.goodreads.com/author/show/27069780...,0.00,['435'],[],"['Published May 26, 2021']","['435 pages, Hardcover']","{'5': '0', '4': '0', '3': '0', '2': '0', '1': ...",https://images-na.ssl-images-amazon.com/images...,"This study focuses on all of Marivaux's works,..."
69,9783319480886,https://www.goodreads.com/search?q=9783319480886,Plants and Health: New Perspectives on the Hea...,Elizabeth Anne Olson,https://www.goodreads.com/author/show/7928001....,0.00,['194'],[],"['Published December 29, 2016']","['194 pages, Kindle Edition']","{'5': '0', '4': '0', '3': '0', '2': '0', '1': ...",https://images-na.ssl-images-amazon.com/images...,This volume showcases current ethnobiological ...
70,9780664241629,https://www.goodreads.com/search?q=9780664241629,Augustine: Earlier Writings,Augustine of Hippo,https://www.goodreads.com/author/show/6819578....,3.81,['420'],[],"['First published January 1, 396']","['420 pages, Paperback']","{'5': '13', '4': '18', '3': '11', '2': '4', '1...",https://images-na.ssl-images-amazon.com/images...,These are Augustine's writings from the time o...


## ISBNDB Data

In [55]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd
from pandas import json_normalize
import requests as req

In [56]:
load_dotenv()
API_KEY = os.getenv('API_KEY')
BASE_URL = "https://api2.isbndb.com/book/"

In [57]:
h = {
    'Authorization': API_KEY
}

isbndb = []


In [99]:
isbn_df.head()

,ISBN-10,ISBN-13,goob_ISBN-13,goob_Title,goob_Authors,goob_Publisher,goob_Year,goob_Language,openl_ISBN-13,openl_Title,openl_Authors,openl_Publisher,openl_Year,openl_Language,wiki_ISBN-13,wiki_Title,wiki_Authors,wiki_Publisher,wiki_Year,wiki_Language
9575640,1534664726,9781534664722.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21158276,0815628129,9780815628125.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19786266,1794862188,9781794862180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37931203,8494761102,9788494761102.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23836774,2900768632,9782900768631.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
isbn_10

['1534664726',
 '0815628129',
 '1794862188',
 '8494761102',
 '2900768632',
 '059081933X',
 '9864855433',
 '8416428573',
 '6587043232',
 '2711833828',
 '0191531979',
 '0963456741',
 '3552033300',
 '1546215271',
 '7301136293',
 '1317363485',
 '8416094667',
 '0500342954',
 '1644928361',
 '6053324493',
 '3788706643',
 '0618591141',
 '1139087193',
 '8981075166',
 '1410814718',
 '044464105X',
 '0072831537',
 '0415592313',
 '8172201478',
 '0312988117',
 '1015359329',
 '1401394426',
 '0070546444',
 '101355891X',
 '1305039947',
 '1461052661',
 '3930612577',
 '108131639X',
 '0342099973',
 '1581180780',
 '1511495219',
 '085045185X',
 '0711038325',
 '1475776012',
 '1398604100',
 '1451597797',
 '0692672109',
 '0618934669',
 '1448895472',
 '1322871051',
 '8495119021',
 '0739478842',
 '1304784754',
 '1791326609',
 '1622296613',
 '0821342169',
 '0701124326',
 '970722357X',
 '1987480554',
 '0413753301',
 '9174962450',
 '1134733054',
 '9040200580',
 '8571772134',
 '3509007875',
 '1349725064',
 '08137245

In [60]:
for isbn in isbn_10:
    isbn_str = str(isbn)  # Convert ISBN to string
    resp = req.get("https://api2.isbndb.com/book/" + isbn_str, headers=h)
    if resp.status_code == 200:
        isbndb.append(resp.json())
    elif resp.status_code == 404:
        print(f"ISBN {isbn_str} not found. Error 404: {resp.text}")
    else:
        print(f"Error {resp.status_code}: {resp.text}")

ISBN 1534664726 not found. Error 404: {"errorType":"string","errorMessage":"Not Found","trace":[]}
ISBN 1794862188 not found. Error 404: {"errorType":"string","errorMessage":"Not Found","trace":[]}
ISBN 9864855433 not found. Error 404: {"errorType":"string","errorMessage":"Not Found","trace":[]}
ISBN 7301136293 not found. Error 404: {"errorType":"string","errorMessage":"Not Found","trace":[]}
ISBN 1317363485 not found. Error 404: {"errorType":"string","errorMessage":"Not Found","trace":[]}
ISBN 1644928361 not found. Error 404: {"errorType":"string","errorMessage":"Not Found","trace":[]}
ISBN 3788706643 not found. Error 404: {"errorType":"string","errorMessage":"Not Found","trace":[]}
ISBN 8981075166 not found. Error 404: {"errorType":"string","errorMessage":"Not Found","trace":[]}
ISBN 1410814718 not found. Error 404: {"errorType":"string","errorMessage":"Not Found","trace":[]}
ISBN 1015359329 not found. Error 404: {"errorType":"string","errorMessage":"Not Found","trace":[]}
ISBN 10135

In [61]:
print(isbndb)

[{'book': {'publisher': 'Syracuse University Press', 'synopsis': 'The urgency to tell the story of the former Yugoslav Republic of Macedonia, the only republic in the former Yugoslavia to secede without bloodshed, is made more compelling by the crisis in Kosovo. In Making Peace Prevail, Alice Ackermann offers the first in-depth account of how Macedonia—one of the few examples of successful preventive diplomacy—held onto peace during the violent breakup of Yugoslavia in the 1990s. Faced with ethnic tensions and the threat of the Bosnian war, this republic was spared the fate of Croatia and Bosnia. With this book Ackermann furthers our understanding of the challenge in conflict prevention in multiethnic and newly democratized societies. She provides a framework of analysis that underscores the "art of conflict prevention." She notes the activity of the major players such as the United Nations and the Organization for Security and Cooperation in Europe (OSCE) but maintains that groups suc

In [62]:
isbn_data = json_normalize(isbndb, sep='_')
isbn_data

,book_publisher,book_synopsis,book_language,book_image,book_title_long,book_edition,book_dimensions,book_dimensions_structured_length_unit,book_dimensions_structured_length_value,book_dimensions_structured_width_unit,...,book_authors,book_title,book_isbn13,book_msrp,book_binding,book_isbn,book_isbn10,book_related_Hardcover,book_other_isbns,book_related_eBook
0,Syracuse University Press,The urgency to tell the story of the former Yu...,en,https://images.isbndb.com/covers/1049363348247...,Making Peace Prevail: Preventing Violent Confl...,First Edition,"Height: 9.01998196 Inches, Length: 6.11998776 ...",inches,6.119988,inches,...,[Alice Ackermann],Making Peace Prevail: Preventing Violent Confl...,9780815628125,11.13,Hardcover,0815628129,0815628129,NaN,NaN,NaN
1,Editorial Samaritana,NaN,es,https://images.isbndb.com/covers/1527354348521...,LATIDOS DE VALDEDIOS,1,"Height: 0.82677 inches, Length: 0.62992 inches...",inches,0.629920,inches,...,[CSCJ Madre Olga María del Redentor],LATIDOS DE VALDEDIOS,9788494761102,0.00,Paperback,8494761102,8494761102,NaN,NaN,NaN
2,Le Dé bleu,Text: French,fr,https://images.isbndb.com/covers/2224251348322...,Dizains disette (French Edition),NaN,NaN,NaN,NaN,NaN,...,[Jean-Pierre Georges],Dizains disette (French Edition),9782900768631,0.00,Paperback,2900768632,2900768632,NaN,NaN,NaN
3,Scholastic Inc.,This series features timeless stories that are...,en,https://images.isbndb.com/covers/9268943482398...,The Tale of Peter Rabbit,NaN,Weight: 0.2 Pounds,NaN,NaN,NaN,...,[Beatrix Potter],The Tale of Peter Rabbit,9780590819336,0.00,Paperback,059081933X,059081933X,NaN,NaN,NaN
4,Yermo ediciones,NaN,es,https://images.isbndb.com/covers/1835035348518...,"METABARON 01. WILHELM-100, EL TECNOALMIRANTE",NaN,"Height: 11.023622036 Inches, Length: 7.2834645...",inches,7.283465,inches,...,"[Alejandro/ Frissen, Jerry/ Secher, Valentin J...","METABARON 01. WILHELM-100, EL TECNOALMIRANTE",9788416428571,0.00,Hardcover,8416428573,8416428573,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,Penguin Books,In a book which uniquely captures 1950's Ameri...,en,https://images.isbndb.com/covers/2533996348223...,I Remember,Reprint,"Height: 7.75 Inches, Length: 5.25 Inches, Weig...",inches,5.250000,inches,...,[Joe Brainard],I Remember,9780140245219,9.95,Paperback,0140245219,0140245219,NaN,NaN,NaN
66,Springer Nature,This volume showcases current ethnobiological ...,English,https://images.isbndb.com/covers/2458235348336...,NaN,2,Page Fidelity,NaN,NaN,NaN,...,"[Marco Ariola, Alfredo Pironti]",Plants and Health,9783319480886,89.99,eBook,331948088X,331948088X,3319480863,"[{'isbn': '9783319480862', 'binding': 'print'}]",NaN
67,Westminster John Knox Press,These are Augustine's writings from the time o...,en,https://images.isbndb.com/covers/1326133348242...,Augustine: Earlier Writings (The Library of Ch...,Reprint,"Height: 8.5 Inches, Length: 5.5 Inches, Weight...",inches,5.500000,inches,...,[J. H. S. Burleigh],Augustine: Earlier Writings (The Library of Ch...,9780664241629,29.95,Paperback,066424162X,066424162X,NaN,"[{'isbn': '9781646980451', 'binding': 'printis...",NaN
68,Icon Group Intl Inc,"On the demand side, exporters and strategic pl...",en,https://images.isbndb.com/covers/2236493348240...,The 2000 Import and Export Market for Internal...,NaN,NaN,NaN,NaN,NaN,...,[Inc Icon Group International],The 2000 Import and Export Market for Internal...,9780597746093,325,Paperback,0597746095,0597746095,NaN,NaN,NaN
